In [1]:
%display latex

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os

In [3]:
eps = var('eps', latex_name = '\\varepsilon')
t = var('t')
r = var('r')
th = var('th', latex_name = '\\theta')
ph = var('ph', latex_name = '\\varphi')

coords = (t,r,th,ph)

h = function('h')(r,th)
k = function('k')(r,th)
m = function('m')(r,th)
omega = function('omega', latex_name='\\omega')(r,th)

nu = function('nu', latex_name='\\nu')(r)
lamb = function('lamb', latex_name='\\lambda')(r)

Omegae = function('Omegaevalue')(eps)
Ne = function('Ne', latex_name='N_e')(r,eps)

In [4]:
def cosmotensors(gedd):
    
    geuu = 1/gedd
    
    chris = [[[0 for k in range(0,4)] for j in range(0,4)] for i in range(0,4)]
    for i in range(0,4):
        for j in range(0,4):
            for k in range(0,4):
                chris[i][j][k] = 0.5*sum(geuu[i,dd]*(derivative(gdd[dd,j],coords[k])
                                                  + derivative(gdd[dd,k],coords[j])
                                                  - derivative(gdd[j,k],coords[dd])) for dd in range(0,4))
                
    riem = [[[[0 for l in range(0,4)] for k in range(0,4)] for j in range(0,4)] for i in range(0,4)]
    for i in range(0,4):
        for j in range(0,4):
            for k in range(0,4):
                for l in range(0,4):
                    riem[i][j][k][l] = (derivative(chris[i][j][l],coords[k])
                                      - derivative(chris[i][j][k],coords[l])
                                      + sum(chris[i][k][dd]*chris[dd][l][j]
                                          - chris[i][l][dd]*chris[dd][k][j] for dd in range(0,4)))
                    
    ricci = matrix(SR, 4, 4)
    for i in range(0,4):
        for k in range(0,4):
            ricci[i,k] = sum(riem[dd][i][dd][k] for dd in range(0,4))
            
    s_curv = sum(sum(ricci[i,j]*geuu[i,j] for i in range(0,4)) for j in range(0,4))
    
    eins = matrix(SR, 4, 4)
    eins = ricci - 0.5*gedd*s_curv
    
    deins = matrix(SR, 4, 4)
    ddeins = matrix(SR, 4, 4)
    for i in range(0,4):
        for j in range(0,4):
            deins[i,j] = diff(eins[i,j],eps)
            ddeins[i,j] = diff(eins[i,j],eps,eps)
    
    return {'inverse_metric': geuu,
            'einstein': eins,
            'd_einstein': deins,
            'dd_einstein': ddeins}

In [5]:
start_time = time.time()

gdd = matrix(SR, 4, 4)

gdd[0,0] = -exp(nu)
gdd[1,1] = exp(lamb)
gdd[2,2] = r^2
gdd[3,3] = r^2*sin(th)^2

guu = 1/gdd

K1 = matrix(SR, 4, 4)
K1[0,3] = -omega*r^2*sin(th)^2
K1[3,0] = -omega*r^2*sin(th)^2

K2 = matrix(SR, 4, 4)
K2[0,0] = -4*exp(nu)*h + 2*r^2*sin(th)^2*omega^2
K2[1,1] = 4*exp(lamb)*m
K2[2,2] = 4*k*r^2
K2[3,3] = 4*k*r^2*sin(th)^2

gedd = matrix(SR, 4, 4)
for i in range(0,4):
    for j in range(0,4):
        gedd[i,j] = gdd[i,j] + eps*K1[i,j] + 1/2*eps^2*K2[i,j]

tensors = cosmotensors(gedd)

geuu = tensors['inverse_metric']
eins = tensors['einstein']                                                                               #G_\mu\nu e
deins = tensors['d_einstein']
ddeins = tensors['dd_einstein']

print("--- %s seconds ---" % (time.time() - start_time))

--- 5.804556846618652 seconds ---


In [6]:
# Para imprimir "más bonito":

show("ge0 = ", gdd[:])
show("K1 = ", K1[:])
show("K2 = ", K2[:])
show("ge = ", gedd[:])

'ge0 = ' [     -e^nu(r)             0             0             0]
[            0     e^lamb(r)             0             0]
[            0             0           r^2             0]
[            0             0             0 r^2*sin(th)^2]

'K1 = ' [                          0                           0                           0 -r^2*omega(r, th)*sin(th)^2]
[                          0                           0                           0                           0]
[                          0                           0                           0                           0]
[-r^2*omega(r, th)*sin(th)^2                           0                           0                           0]

'K2 = ' [2*r^2*omega(r, th)^2*sin(th)^2 - 4*e^nu(r)*h(r, th)                                                   0                                                   0                                                   0]
[                                                  0                                4*e^lamb(r)*m(r, th)                                                   0                                                   0]
[                                                  0                                                   0                                      4*r^2*k(r, th)                                                   0]
[                                                  0                                                   0                                                   0                            4*r^2*k(r, th)*sin(th)^2]

'ge = ' [(r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*eps^2 - e^nu(r)                                                                   0                                                                   0                                     -eps*r^2*omega(r, th)*sin(th)^2]
[                                                                  0                              2*eps^2*e^lamb(r)*m(r, th) + e^lamb(r)                                                                   0                                                                   0]
[                                                                  0                                                                   0                                          2*eps^2*r^2*k(r, th) + r^2                                                                   0]
[                                    -eps*r^2*omega(r, th)*sin(th)^2                                                                   0                                                                   0                      2*eps^2*r^2*k(r, th)*sin(th)^2 + r^2*sin(th)^2]

In [7]:
#4-velocities of the fluid for any epsilon

ue = [0,0,0,0]
u = [0,0,0,0]

ue[0] = Ne;         ue[1] = 0;   ue[2] = 0;   ue[3] = Ne*Omegae
u[0] = exp(-nu/2);  u[1] =  0;   u[2] = 0;    u[3] =  0

Nevalue = solve(u[0]==ue[0], Ne)[0].rhs()                                                             #Eq 4.20
Omegaevalue = solve(u[3]==ue[3], Omegae)[0].rhs()                                                     #Eq 4.20

In [8]:
show(LatexExpr("N_{\\varepsilon}\lvert_0 = "), Nevalue)
show(LatexExpr("\\Omega_{\\varepsilon}\lvert_0 = "), Omegaevalue)

N_{\varepsilon}\lvert_0 =  e^(-1/2*nu(r))

\Omega_{\varepsilon}\lvert_0 =  0

In [13]:
u1 = [0,0,0,0]
u2 = [0,0,0,0]

u10 = function('u10',latex_name="u^{(1)0}")(r)
u20 = function('u20',latex_name="u^{(2)0}")(r)

Omega1 = function('Omega1', latex_name="\\Omega^{(1)}")(eps)
Omega2 = function('Omega2', latex_name="\\Omega^{(2)}")(eps)

s0 = eps == 0
s1 = Ne.subs(eps==0) == Nevalue
s2 = Omegae.subs(eps==0) == Omegaevalue
s3 = diff(Ne,eps).subs(eps==0) == u10
s4 = diff(Omegae,eps).subs(eps==0) == Omega1
s5 = diff(Ne,eps,eps).subs(eps==0) == u20
s6 = diff(Omegae,eps,eps).subs(eps==0) == Omega2

sustituciones = (s0,s1,s2,s3,s4,s5,s6)

for i in range(0,4):
    u1[i] = diff(ue[i],eps).subs(sustituciones)
    u2[i] = diff(ue[i],eps,eps).subs(sustituciones)

Ue = [0,0,0,0]
for i in range(0,4):
    for j in range(0,4):
        Ue[i] += ue[j]*gedd[j,i]

geUeue = 0
for i in range(0,4):
    geUeue += Ue[i]*ue[i]

dgeUeue = diff(geUeue, eps).subs(sustituciones) == 0
u10value = solve(dgeUeue, u10)[0].rhs()
s7 = u10 == u10value
for i in range(0,4):
    u1[i] = u1[i].subs(s7)                                                                                 #Eq 4.21

ddgeUeue = diff(geUeue, eps, eps).subs(sustituciones) == 0
u20value = solve(ddgeUeue, u20)[0].rhs()
s8 = u20 == u20value
for i in range(0,4):
    u2[i] = u2[i].subs(u20==u20value).subs(u10==u10value)                                                  #Eq 4.22

In [14]:
show(LatexExpr("u^{(1)} = "), u1)
show(LatexExpr("u^{(2)} = "), u2)

u^{(1)} =  [0, 0, 0, Omega1(eps)*e^(-1/2*nu(r))]

u^{(2)} =  [((r^2*Omega1(eps)^2 - 2*r^2*Omega1(eps)*omega(r, th) + r^2*omega(r, th)^2)*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-3/2*nu(r)),
 0,
 0,
 Omega2(eps)*e^(-1/2*nu(r))]

In [15]:
for i in range(0,4):
    for j in range(0,4):
        deins[i,j] = deins[i,j].subs(sustituciones)                                             #d/de(G_\mu\nu)|e=0
        ddeins[i,j] = ddeins[i,j].subs(sustituciones)                                       #d^2/de^2(G_\mu\nu)|e=0

In [16]:
Ee = function('Ee', latex_name='E_{\\varepsilon}')(r,eps)
Pe = function('Pe', latex_name='P_{\\varepsilon}')(r,eps)

eins0 = matrix(SR, 4, 4)
Tedd = matrix(SR, 4, 4)
Tdd = matrix(SR, 4, 4)
for i in range(0,4):
    for j in range(0,4):
        eins0[i,j] = eins[i,j].subs(sustituciones)                                                         #G_\mu\nu
        Tedd[i,j] = (Ee + Pe)*Ue[i]*Ue[j] + Pe*gedd[i,j]                                                 #T_\mu\nu e
        Tdd[i,j] = Tedd[i,j].subs(sustituciones)                                                           #T_\mu\nu

In [17]:
show(LatexExpr("T_{\\varepsilon}\lvert_0 ="), Tdd)

T_{\varepsilon}\lvert_0 = [(Ee(r, 0) + Pe(r, 0))*e^nu(r) - Pe(r, 0)*e^nu(r)                                                0                                                0                                                0]
[                                               0                               Pe(r, 0)*e^lamb(r)                                                0                                                0]
[                                               0                                                0                                     r^2*Pe(r, 0)                                                0]
[                                               0                                                0                                                0                           r^2*Pe(r, 0)*sin(th)^2]

In [18]:
kappa = 8*pi
E = function('E')(r,th)
P = function('P')(r,th)

s9 = Ee.subs(eps==0) == E
s10 = Pe.subs(eps==0) == P
sustituciones = (s1,s2,s3,s4,s5,s6,s7,s8,s9,s10)
for i in range(0,4):
    for j in range(0,4):
        Tdd[i,j] = Tdd[i,j].subs(sustituciones)
E_tt = solve(eins0[0,0]==kappa*Tdd[0,0],E)[0].rhs()                                                          #Eq 3.7
P_rr = solve(eins0[1,1]==kappa*Tdd[1,1],P)[0].rhs()                                                          #Eq 3.8
P_thth = solve(eins0[2,2]==kappa*Tdd[2,2],P)[0].rhs()
P_phph = solve(eins0[3,3]==kappa*Tdd[3,3],P)[0].rhs()
ddnu = solve(P_rr == P_thth, diff(nu,r,r))[0].rhs()                                                          #Eq 3.9

In [19]:
show(LatexExpr("E = "), E_tt)
show(LatexExpr("P_{rr} = "), P_rr)
show(LatexExpr("\\nu'' ="), ddnu)

E =  1/8*(r*e^(-lamb(r))*diff(lamb(r), r) - e^(-lamb(r)) + 1)/(pi*r^2)

P_{rr} =  1/8*(r*e^(-lamb(r))*diff(nu(r), r) + e^(-lamb(r)) - 1)/(pi*r^2)

\nu'' = -1/2*(r^2*diff(nu(r), r)^2 - 2*r*diff(lamb(r), r) - (r^2*diff(lamb(r), r) + 2*r)*diff(nu(r), r) + 4*e^lamb(r) - 4)/r^2

In [20]:
dlamb = solve(E-E_tt==0,diff(lamb,r))[0].rhs()                                                              #Eq 3.10
dnu = solve(P-P_rr==0,diff(nu,r))[0].rhs()                                                                  #Eq 3.11

In [21]:
show(LatexExpr("\\lambda' = "), dlamb)
show(LatexExpr("\\nu' = "), dnu)

\lambda' =  (8*pi*r^2*E(r, th)*e^lamb(r) - e^lamb(r) + 1)/r

\nu' =  (8*pi*r^2*P(r, th)*e^lamb(r) + e^lamb(r) - 1)/r

In [22]:
h0 = function('h0', latex_name='h_0')(r,th)
k0 = function('k0', latex_name='k_0')(r,th)
m0 = function('m0', latex_name='m_0')(r,th)

h1 = function('h1', latex_name='h_1')(r,th)
k1 = function('k1', latex_name='k_1')(r,th)
m1 = function('m1', latex_name='m_1')(r,th)

h2 = function('h2', latex_name='h_2')(r,th)
k2 = function('k2', latex_name='k_2')(r,th)
m2 = function('m2', latex_name='m_2')(r,th)

LP1 = function('LP1', latex_name='LP_1')(th)
LP2 = function('LP2', latex_name='LP_2')(th)

E1 = function('E1', latex_name='E_1')(r)
E2 = function('E2', latex_name='E_2')(r)
P1 = function('P1', latex_name='P_1')(r)
P2 = function('P2', latex_name='P_2')(r)

In [23]:
s11 = diff(Ee,eps).subs(eps==0) == E1
s12 = diff(Ee,eps,eps).subs(eps==0) == E2
s13 = diff(Pe,eps).subs(eps==0) == P1
s14 = diff(Pe,eps,eps).subs(eps==0) == P2

sustituciones = (s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14)

In [24]:
dTdd = matrix(SR, 4, 4)
ddTdd = matrix(SR, 4, 4)
for i in range(0,4):
    for j in range(0,4):
        dTdd[i,j] = diff(Tedd[i,j],eps)
        ddTdd[i,j] = diff(Tedd[i,j],eps,eps)
    
for i in range(0,4):
    for j in range(0,4):
        dTdd[i,j] = dTdd[i,j].subs(sustituciones)                                               #d/de(T_\mu\nu)|e=0
        ddTdd[i,j] = ddTdd[i,j].subs(sustituciones)                                         #d^2/de^2(T_\mu\nu)|e=0

In [25]:
# Calculamos ahora la primera derivada de la ecuación

dEq11 = deins[1,1] - kappa*dTdd[1,1] == 0
dEq11 = dEq11.simplify_full()

dEq22 = deins[2,2] - kappa*dTdd[2,2] == 0
dEq22 = dEq22.simplify_full()

dEq33 = deins[3,3] - kappa*dTdd[3,3] == 0
dEq33 = dEq33.simplify_full()

show(dEq11)
show(dEq22)
show(dEq33)

P1value = solve(dEq11,P1)[0].rhs()

show(LatexExpr("\\text{De aquí sacamos }P_1(r) \\rightarrow P_1(r) = "), P1value)

-8*pi*P1(r)*e^lamb(r) == 0

-8*pi*r^2*P1(r) == 0

-8*pi*r^2*P1(r)*sin(th)^2 == 0

\text{De aquí sacamos }P_1(r) \rightarrow P_1(r) =  0

In [26]:
dEq00 = deins[0,0] - kappa*dTdd[0,0] == 0
dEq00 = dEq00.simplify_full()
show(dEq00)

E1value = solve(dEq00.subs(sustituciones),E1)[0].rhs()
show(LatexExpr("\\text{De aquí sacamos }E_1(r,\\theta) \\rightarrow E_1(r,\\theta) = "), E1value)

-16*pi*E(r, th)*e^(3/2*nu(r))*u10(r) - 16*pi*P(r, th)*e^(3/2*nu(r))*u10(r) - 8*pi*E1(r)*e^nu(r) == 0

\text{De aquí sacamos }E_1(r,\theta) \rightarrow E_1(r,\theta) =  0

In [40]:
#DEMOSTRAR QUE ESTO ES LA ECUACIÓN 48
dEq03 = deins[0,3] - kappa*dTdd[0,3] == 0
#show(dEq03.simplify_full())
dEq03 = -dEq03
Evalue = diff(lamb,r)/(r*e^(lamb)*8*pi)-(1-e^(lamb))/(r^2*e^(lamb)*8*pi)
Pvalue = diff(nu,r)/(r*e^(lamb)*8*pi)-(e^(lamb)-1)/(r^2*e^(lamb)*8*pi)
dEq03 = dEq03.subs(E==Evalue,P==Pvalue)
dEq03 = dEq03*e^(lamb)/(sin(th)^2)
dEq03 = dEq03.subs(diff(nu,r,r)==ddnu)
dEq03 = dEq03.simplify_full()
dEq03                                                                                                        #Eq 48

(-1.0*e^lamb(r) + 1.0)*omega(r, th) + (-1.0*r*Omega1(eps) + 0.5*r*omega(r, th))*diff(lamb(r), r) + (-1.0*r*Omega1(eps) + 0.5*r*omega(r, th))*diff(nu(r), r) == 0

In [32]:
# Para la segunda derivada tenemos

ddEq00 = ddeins[0,0] - kappa*ddTdd[0,0] == 0
ddEq00 = ddEq00*(r^2*sin(th)^4)
ddEq00 = ddEq00.simplify_full()
ddEq00

1/2*(4*r^2*cos(th)*e^lamb(r)*omega(r, th)*sin(th)^5*diff(omega(r, th), th) - (r^4*omega(r, th)^2*diff(lamb(r), r)*diff(nu(r), r) - r^4*omega(r, th)^2*diff(nu(r), r)^2 - 2*r^4*omega(r, th)^2*diff(nu(r), r, r) + 6*r^3*omega(r, th)^2*diff(lamb(r), r) - 4*r^3*omega(r, th)*diff(omega(r, th), r) - 4*(16*pi*r^4*E(r, th)*e^lamb(r) + 8*pi*r^4*P(r, th)*e^lamb(r) - 3*r^2*e^lamb(r) + 3*r^2)*omega(r, th)^2 + 64*(pi*r^4*E(r, th)*Omega1(eps)*e^lamb(r) + pi*r^4*Omega1(eps)*P(r, th)*e^lamb(r))*omega(r, th))*sin(th)^6 + 8*cos(th)*e^(lamb(r) + nu(r))*sin(th)^3*diff(k(r, th), th) - 2*(r^2*e^nu(r)*m(r, th)*diff(lamb(r), r)*diff(nu(r), r) - r^2*e^nu(r)*h(r, th)*diff(nu(r), r)^2 + 8*pi*r^2*E2(r)*e^(lamb(r) + nu(r)) - 2*r^2*e^lamb(r)*omega(r, th)^2 - r^2*e^nu(r)*diff(h(r, th), r)*diff(nu(r), r) + 8*(e^lamb(r) - 1)*e^nu(r)*k(r, th) - 4*r*e^nu(r)*diff(k(r, th), r) + 16*(pi*r^2*e^(lamb(r) + 2*nu(r))*u10(r)^2 + pi*r^2*e^(lamb(r) + 3/2*nu(r))*u20(r))*E(r, th) + 16*(pi*r^2*e^(lamb(r) + 2*nu(r))*u10(r)^2 + pi*r^2*e^(lamb(r) + 3/2*nu(r))*u20(r))*P(r, th) + 4*(16*pi*r^2*E(r, th)*e^(lamb(r) + nu(r)) + 8*pi*r^2*P(r, th)*e^(lamb(r) + nu(r)) - (e^lamb(r) - 1)*e^nu(r))*h(r, th) - 4*e^nu(r)*m(r, th) + 32*(pi*r^2*E1(r)*e^(lamb(r) + 3/2*nu(r)) + pi*r^2*P1(r)*e^(lamb(r) + 3/2*nu(r)))*u10(r) - 4*(r*e^nu(r)*h(r, th) - r*e^nu(r)*k(r, th) - r*e^nu(r)*m(r, th))*diff(lamb(r), r) + (r^2*e^nu(r)*diff(nu(r), r) - 4*r*e^nu(r))*diff(m(r, th), r))*sin(th)^4)*e^(-lamb(r)) == 0

In [33]:
ddEq03 = ddeins[0,3] - kappa*ddTdd[0,3] == 0
ddEq03 = ddEq03*(3/(16*pi*r^2*Omega2))
ddEq03 = ddEq03.simplify_full()
ddEq03

3/2*(2*E1(r)*Omega1(eps)*e^(1/2*nu(r)) + 2*Omega1(eps)*P1(r)*e^(1/2*nu(r)) + (4*Omega1(eps)*e^nu(r)*u10(r) + Omega2(eps)*e^(1/2*nu(r)))*E(r, th) + (4*Omega1(eps)*e^nu(r)*u10(r) + Omega2(eps)*e^(1/2*nu(r)))*P(r, th) - 2*(2*E(r, th)*e^nu(r)*u10(r) + 2*P(r, th)*e^nu(r)*u10(r) + E1(r)*e^(1/2*nu(r)))*omega(r, th))*e^(-1/2*nu(r))*sin(th)^2/Omega2(eps) == 0

In [34]:
ddEq12 = ddeins[1,2] - kappa*ddTdd[1,2] == 0
ddEq12 = ddEq12*(r*e^(nu))
ddEq12 = ddEq12.simplify_full()
ddEq12

4.0*r^2*omega(r, th)*sin(th)^2*diff(omega(r, th), th) + 2.0*r*e^nu(r)*diff(lamb(r), r)*diff(m(r, th), th) + 2.0*r*e^nu(r)*diff(h(r, th), th)*diff(nu(r), r) + (1.0*r^3*omega(r, th)^2*diff(nu(r), r) + 6.0*r^2*omega(r, th)^2)*cos(th)*sin(th) + 4.0*e^nu(r)*diff(k(r, th), th) == 0

In [35]:
# Aplicamos barotropic EoS
# Utilizando (6.11) de Reina con P1=0 y E1=0
E2*diff(P,r) - P2*diff(E,r) == 0

-P2(r)*diff(E(r, th), r) + E2(r)*diff(P(r, th), r) == 0

In [36]:
for i in range(0,4):
    for j in range(0,4):
        ddeins[i,j] = ddeins[i,j].subs(m==m0+m1*LP1+m2*LP2,
                                       h==m0+h1*LP1+h2*LP2,
                                       k==k0+k1*LP1+k2*LP2)
        ddTdd[i,j] = ddTdd[i,j].subs(m==m0+m1*LP1+m2*LP2,
                                     h==m0+h1*LP1+h2*LP2,
                                     k==k0+k1*LP1+k2*LP2)

In [ ]:
s1 = sin(th)^2 == (2-2*LP2)/3
s2 = 1/sin(th)^2 == 3/(2-2*LP2)
s3 = cos(th) == LP1
s4 = cos(th)^2 == (2*LP2+1)/3
s5 = LP1^2 == (2*LP2+1)/3
s6 = diff(LP1, th, th) == -LP1
s7 = diff(LP2, th) == -3*LP1*sin(th)
s8 = diff(LP2, th, th) == (2-2*LP2)-2*LP2-1
s9 = E1 == E1value
s10 = P1 == P1value
s11 = u10 == u10value
#s12 = sin(th) == - diff(LP1, th)

sustituciones = (s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12)

In [ ]:
for i in range(0,4):
    for j in range(0,4):
        ddeins[i,j] = ddeins[i,j].subs(sustituciones)
        ddTdd[i,j] = ddTdd[i,j].subs(sustituciones)

In [ ]:
for i in range(0,4):
    for j in range(0,4):
        ddeins[i,j] = ddeins[i,j].subs(LP1==0,LP2==0)
        ddTdd[i,j] = ddTdd[i,j].subs(LP1==0,LP2==0)

In [ ]:
#Hacer lo que me dijo Raul de: La derivada de LP2 respecto a theta, es coseno por LP2, que es LP1 por P2 y así.
#O sea pasar todos los theta a LP y todas las derivadas respecto a theta modificarlas
#Y luego sacar las ecuaciones